<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/%E9%81%BA%E4%BC%9D%E5%AD%90_%E7%B5%9E%E3%82%8A%E8%BE%BC%E3%81%BF_%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

In [ ]:
import time
import pandas as pd
from Bio import Entrez

# エクセルファイルから特定（これだと3行目）の列を読み込む
df = pd.read_excel("/content/GV_MⅡ_GSE75738.xls.xlsx", usecols=[1], engine='openpyxl')

# "Name_GeneSymbol" 列を遺伝子名のリストに設定
gene_names = df["Name_GeneSymbol"].tolist()

# NCBIに接続するためのメールアドレスを設定
Entrez.email = "tarumi.wataru@gmail.com"

# 検索するキーワードのリストを指定
search_keywords = ["receptor"]  # 複数のキーワードをリストで指定, ["receptor","lignad"]

# リクエストの間隔（秒）を設定
request_interval = 3  # 3秒ごとにリクエストを送信

# 結果を格納するための空のリスト
results = []

# 遺伝子名を検索し、関連する遺伝子をリストに追加
for gene_name in gene_names:
    search_term = f"{gene_name}[Gene Name]"
    handle = Entrez.esearch(db="gene", term=search_term)
    record = Entrez.read(handle)
    gene_id_list = record["IdList"]

    if gene_id_list:
        gene_id = gene_id_list[0]
        gene_summary = Entrez.esummary(db="gene", id=gene_id)
        try:
            gene_info = Entrez.read(gene_summary)
            description = gene_info[0].get("Description", "情報なし")
            # 遺伝子情報を結果に追加する前にキーワードの存在を確認
            if any(keyword in description for keyword in search_keywords):
                results.append({"Name_GeneSymbol": gene_name, "機能の説明": description})
        except (IndexError, KeyError):
            # エラーメッセージを表示する代わりに、処理をスキップ
            pass
    else:
        # エラーメッセージを表示する代わりに、処理をスキップ
        pass

    # リクエストの間隔を設定
    time.sleep(request_interval)

# 結果をDataFrameに変換
result_df = pd.DataFrame(results)

# ヘッダーを指定して結果をエクセルファイルに書き込む
result_df.to_excel("gene_results.xlsx", index=False, header=["Name_GeneSymbol", "機能の説明"])

KeyboardInterrupt: ignored